# Notebook #1


### Name: Tulsi Patel


### Date: 11.11.2021


### PID: 730392259


### Goal: Find average firing rate for Brain Observatory, Pvalb-IRES-Cre, Female, VISp, Static Gratings (SG) and Drifting Gratings (DG). 

## Protocol

### Starting code. 

In [1]:
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

In [2]:
data_directory = '/Users/tulsipatel/local1/ecephys_cache_dir' 
# must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

In [3]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
units = cache.get_units()

In [5]:
sessions = cache.get_session_table()


print('Total number of sessions: ' + str(len(sessions)))

sessions.head()

Total number of sessions: 58


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [11]:
filtered_sessions = sessions[(sessions.sex == 'F') & \
                             (sessions.full_genotype.str.find('Pvalb-IRES-Cre') > -1) & \
                             (sessions.session_type == 'brain_observatory_1.1') & \
                             (['VISp' in acronyms for acronyms in 
                               sessions.ecephys_structure_acronyms])]

filtered_sessions.head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
746083955,2019-10-03T00:00:00Z,726170935,brain_observatory_1.1,98.0,F,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,582,2216,6,"[VPM, TH, LGd, CA3, CA2, CA1, VISal, nan, grey..."
773418906,2019-10-03T00:00:00Z,757329624,brain_observatory_1.1,124.0,F,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,546,2232,6,"[PPT, NOT, SUB, ProS, CA1, VISam, nan, APN, DG..."


In [14]:
session_id = filtered_sessions.index.values[0]

session = cache.get_session_data(session_id)

In [13]:
units = session.units

print('Total number of units: ' + str(len(units)))

units.head()

/Users/tulsipatel/.local/lib/python3.8/site-packages/allensdk/brain_observatory/ecephys/ecephys_project_api/ecephys_project_warehouse_api.py:291: FutureWarning: Conversion of the second argument of issubdtype from `bool` to `np.generic` is deprecated. In future, it will be treated as `np.bool_ == np.dtype(bool).type`.
  pv_is_bool = np.issubdtype(output["p_value_rf"].values[0], np.bool)


Total number of units: 582


,peak_channel_id,waveform_halfwidth,nn_hit_rate,waveform_amplitude,firing_rate,waveform_velocity_below,cumulative_drift,waveform_spread,amplitude_cutoff,waveform_recovery_slope,max_drift,L_ratio,local_index_unit,waveform_velocity_above,snr,nn_miss_rate,waveform_duration,presence_ratio,isolation_distance,isi_violations,d_prime,waveform_repolarization_slope,silhouette_score,waveform_PT_ratio,cluster_id,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951878213,849740502,0.151089,1.000,252.926895,28.717870,NaN,434.14,30.0,7.133207e-04,-0.127150,36.51,0.000004,0,-0.343384,4.780400,0.000000,0.206030,0.97,225.716963,0.000188,6.730306,0.627684,NaN,0.863448,0,NaN,0.0,3.370224,0.973333,2.664833,0.866667,2.017117,0.121025,0.114119,0.060497,0.015037,NaN,1.096409,0.700,0.08,4.0,0.202559,0.809524,0.377341,-1.000000,0.276139,135.0,30.0,0.158799,0.442171,2.761215e-04,0.166087,0.010034,NaN,4923,0.50,26.480777,0.053289,42.234275,0.042763,39.524292,NaN,0.1195,False,False,False,False,False,0.033915,False,0.051605,1.000000,0.004840,0.864463,0.007788,40,27,760647905,3,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.971918,1249.99883,True
951878313,849740524,0.164824,1.000,183.232725,49.122678,-0.412060,197.05,60.0,1.652670e-02,-0.162992,25.53,0.000003,8,0.343384,2.945178,0.000000,0.274707,0.99,155.205812,0.064869,7.590752,0.757797,0.152861,0.663130,8,NaN,200.0,8.546237,2.338352,1.891111,1.838703,1.708333,0.151282,0.031943,0.019111,0.032959,75.000,0.342966,0.379,0.08,1.0,0.385214,0.482870,0.396450,0.301913,0.361438,315.0,60.0,0.000154,0.000137,1.118768e-07,0.013595,0.000005,-30.000,4969,0.00,43.646607,42.258456,48.903658,72.895138,46.091468,NaN,0.0915,False,False,False,False,False,-0.016881,False,0.008061,0.002490,0.004696,0.007646,0.005235,160,59,760647905,14,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.971918,1249.99883,True
951878350,849740534,0.109883,0.996,149.232525,45.590934,-0.206030,190.89,70.0,1.202701e-03,-0.085155,20.46,0.001864,11,0.206030,3.341322,0.004095,0.206030,0.99,155.571595,0.004993,6.058813,0.636389,0.206435,0.272931,11,NaN,700.0,0.602806,1.037519,1.032773,0.976324,0.927689,0.285332,0.004176,0.006595,0.019456,14.286,0.702162,0.441,0.04,15.0,NaN,-0.241445,-0.226044,-0.217277,-0.192932,0.0,90.0,NaN,0.000292,1.323769e-03,0.605628,0.003479,2.857,4916,0.75,46.779123,47.560746,43.984073,47.308375,43.939267,NaN,0.0255,False,False,False,False,False,-0.108085,False,0.001015,0.000277,0.001427,0.005475,0.002367,200,27,760647905,19,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.971918,1249.99883,True
951878389,849740538,0.123618,0.990,175.529250,14.130594,-0.343384,287.04,70.0,1.216791e-02,-0.151065,21.36,0.000914,14,0.068677,3.166919,0.002185,0.206030,0.99,105.256300,0.029868,4.152507,0.747599,0.093003,0.461972,14,NaN,200.0,16.755080,3.961124,5.045263,7.933333,3.265306,0.356098,0.012501,0.048288,0.090532,10.000,0.894365,0.030,0.02,4.0,0.637658,0.657571,0.793651,0.489130,0.625903,180.0,60.0,0

In [15]:
V1_units = units[(units.ecephys_structure_acronym == 'VISp') &
                (units.isi_violations < 0.1)]

print('Total number of low-contamination V1 units: ' + str(len(V1_units)))


V1_units.head()

Total number of low-contamination V1 units: 8


,peak_channel_id,waveform_halfwidth,nn_hit_rate,waveform_amplitude,firing_rate,waveform_velocity_below,cumulative_drift,waveform_spread,amplitude_cutoff,waveform_recovery_slope,max_drift,L_ratio,local_index_unit,waveform_velocity_above,snr,nn_miss_rate,waveform_duration,presence_ratio,isolation_distance,isi_violations,d_prime,waveform_repolarization_slope,silhouette_score,waveform_PT_ratio,cluster_id,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951888688,849737416,0.109883,1.000000,210.606240,19.822821,0.000000,96.83,60.0,0.000080,-0.215701,59.58,0.000007,255,-0.068677,3.868753,0.000000,0.178559,0.99,174.799716,0.000395,7.829313,0.845139,0.004387,0.525810,262,NaN,200.0,26.106758,2.396347,5.129864,6.302622,4.983624,0.505736,0.082358,0.075863,0.065501,75.000,1.128208,0.159,0.16,1.0,0.616700,0.550802,0.770887,0.793651,0.593072,45.0,30.0,0.040645,0.007535,6.196049e-13,0.012087,0.000118,-30.000,4914,0.00,15.852672,17.212461,30.143541,11.157956,18.543831,NaN,0.1515,False,False,False,False,False,-0.115017,False,0.130855,0.033254,0.030774,0.093679,0.089175,2000,59,760647909,198,385.0,VISp,NaN,NaN,NaN,probeC,See electrode locations,29999.998792,1249.99995,True
951888780,849737454,0.164824,0.997333,104.898105,23.041273,-0.686767,99.96,60.0,0.007313,-0.075667,45.19,0.007306,265,-0.412060,2.324635,0.004750,0.384590,0.99,82.726781,0.006213,4.217935,0.405303,0.066737,0.511508,272,NaN,700.0,6.450985,0.923740,2.497720,2.836183,2.306631,0.299705,0.014646,0.078324,0.089494,20.000,1.143596,0.000,0.04,1.0,0.231608,-0.085322,0.418066,-0.109091,0.206286,180.0,30.0,0.013299,0.342036,8.528009e-06,0.735289,0.063228,10.000,5009,0.75,29.078500,25.472310,20.317806,39.572586,25.619249,NaN,0.0995,False,False,False,False,False,0.344966,False,0.086766,0.001714,0.126981,0.059734,0.088955,2180,11,760647909,217,385.0,VISp,NaN,NaN,NaN,probeC,See electrode locations,29999.998792,1249.99995,True
951888826,849737462,0.151089,0.985955,200.400915,6.787595,0.480737,90.58,60.0,0.000393,-0.133277,39.39,0.004147,271,0.686767,3.624357,0.002869,0.549414,0.99,62.155828,0.014320,3.324987,0.564063,0.069199,0.584534,278,NaN,500.0,1.260841,0.709425,1.170227,0.868817,1.036843,0.384582,0.042403,0.164681,0.097916,44.000,0.397449,0.000,0.32,1.0,0.293333,0.340659,0.329193,0.111111,0.363910,0.0,60.0,0.003582,0.240600,4.389078e-04,0.830681,0.002095,2.000,4992,0.25,6.682278,2.984204,8.710030,2.024134,6.236120,NaN,0.0995,False,False,False,False,False,0.586610,False,0.057373,0.002548,0.147650,0.272095,0.147058,2220,11,760647909,221,385.0,VISp,NaN,NaN,NaN,probeC,See electrode locations,29999.998792,1249.99995,True
951888857,849737464,0.206030,0.944000,254.223450,13.335804,0.206030,107.46,80.0,0.000573,-0.139524,41.93,0.033589,275,0.206030,4.064986,0.010957,0.521943,0.99,49.684503,0.003710,3.650694,0.706108,0.147871,0.521751,282,NaN,500.0,5.246579,1.530575,1.108825,1.788235,1.176619,0.290149,0.057480,0.094983,0.089971,18.000,0.292328,0.000,0.02,1.0,0.381107,0.543182,0.2636

## Find average firing rate for static gratings. 

In [16]:
print(V1_units.firing_rate_sg)

unit_id
951888688    18.543831
951888780    25.619249
951888826     6.236120
951888857     9.775828
951888874     9.627285
951889030    13.249590
951889425     0.205828
951889147    17.043752
Name: firing_rate_sg, dtype: float64


In [18]:
static_firing_rates=V1_units.firing_rate_sg 
static_average_firing_rate = sum(V1_units.firing_rate_sg)/len(V1_units.firing_rate_sg)
print(static_average_firing_rate)

12.537685328900817


## Find average firing rate for drifting gratings. 

In [ ]:
print(V1_units.firing_rate_dg)

In [ ]:
drifting_firing_rates=V1_units.firing_rate_dg 
drifting_average_firing_rate = sum(V1_units.firing_rate_dg)/len(V1_units.firing_rate_dg)
print(drifting_average_firing_rate)

## Remaining code: I was experimenting with another way to find the average firing rate. 

In [ ]:
presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'natural_scenes']


presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values
presentation_table.head()

In [ ]:
units_of_interest = V1_units.index.values

In [ ]:
times = session.presentationwise_spike_times(
    stimulus_presentation_ids=presentation_ids,
    unit_ids=units_of_interest
)

times.head()

In [ ]:
average_firing_rate = '.0023476'

In [ ]:
length_of_time=times.time_since_stimulus_presentation_onset

In [ ]:
print(length_of_time.index.values[4])